# Experiment's Exploratory Data Analisys

Explores the results of the experiments.

In [1]:
% matplotlib inline
import pandas as pd
from pymongo import MongoClient

db = MongoClient().imdbws
df = pd.DataFrame(list(db.exp_0.find())).set_index('_id')
df = df.drop(['title', 'ypct_rating', 'ypct_votes'], axis=1)

c = df[df.columns.drop(labels=['top100', 'year'])].corr(method='spearman')['ypct']
order = c.abs().sort_values(ascending = False).index
c[order].head(15)

ypct                                 1.000000
ego_previous_votes_maximum           0.744202
ego_previous_votes_median            0.742569
ego_previous_votes_mean              0.739110
team_previous_votes                  0.723377
ego_previous_votes_minimum           0.595992
ego_previous_rating_std_dev          0.538119
team_closeness                       0.363530
ego_closeness_minimum                0.352576
ego_closeness_maximum                0.342444
team_size                            0.338990
ego_closeness_mean                   0.336290
pair_shared_collaborators_maximum    0.326189
pair_shared_collaborators_mean       0.319504
ego_closeness_median                 0.314554
Name: ypct, dtype: float64